In [1]:
import PIL
import numpy as np

In [2]:
import matplotlib.pyplot as plt

In [10]:
def halftone(img_arr):
    img = PIL.Image.fromarray(img_arr)
    img = img.convert('1')
    return np.asarray(img)

In [102]:
def main_arr(img_arr, qtree_level=0, x_index=0, y_index=0):
    if qtree_level == 0:
        return halftone(img_arr)
    
    midpoint_y = int(img_arr.shape[0] / 2)
    midpoint_x = int(img_arr.shape[1] / 2)
    lu = img_arr[:midpoint_y, :midpoint_x]
    ru = img_arr[midpoint_y:, :midpoint_x]
    ll = img_arr[:midpoint_y, midpoint_x:]
    rl = img_arr[midpoint_y:, midpoint_x:]

    lut = {
        (0, 0): lu,
        (0, 1): ll,
        (1, 0): ru,
        (1, 1): rl
    }
    
    if qtree_level == 1:
        return halftone(lut[(x_index, y_index)])
    
    if qtree_level > 1:
        qtree_level -= 1
        xi = 1 if x_index & (1 << qtree_level) else 0
        yi = 1 if y_index & (1 << qtree_level) else 0
        print(x_index, y_index, xi, yi)
        return main_arr(lut[(xi, yi)], qtree_level, x_index - (xi << qtree_level), y_index - (yi << qtree_level))

In [103]:
def main(in_fn, qtree_level=0, x_index=0, y_index=0):
    img = PIL.Image.open(in_fn)
    arr = np.asarray(img).copy()
    orig_shape = arr.shape
    transparent_pixels = arr[..., 3]
    arr[transparent_pixels == 0] = [255, 255, 255, 255]
    arr = arr.reshape(orig_shape)
    ret = main_arr(arr, qtree_level, x_index, y_index)
    return PIL.Image.fromarray(ret)

In [113]:
for x in range(4):
    for y in range(4):
        ret = main("./CharaStudio-2021-11-13-16-42-14-Render.png", 2, x, y)
        ret.save("./burn_{}_{}.png".format(y, x))

0 0 0 0
0 1 0 0
0 2 0 1
0 3 0 1
1 0 0 0
1 1 0 0
1 2 0 1
1 3 0 1
2 0 1 0
2 1 1 0
2 2 1 1
2 3 1 1
3 0 1 0
3 1 1 0
3 2 1 1
3 3 1 1


In [114]:
main("./CharaStudio-2021-11-13-16-42-14-Render.png").save("./burn_full.png")